# Analysis of chemo diversity on pharmacological and biological activities

The [dataset](results/activities_2022-01-29_16-33-05.csv) is obtained from the Scopus downloader version 2.0 from the [base set](data/activities.csv) of chemical compounds and activities.

**TODO** check:

- <https://stackoverflow.com/questions/53927460/select-rows-in-pandas-multiindex-dataframe>
- <https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#multiindex-query-syntax>


## Loading data from Scopus

### Boilerplate

Some Python configuration.


In [48]:
from pathlib import Path
from itertools import islice, product
from pprint import pprint

import pandas as pd
import numpy as np
from scipy import linalg

# import prince
import holoviews as hv
import seaborn as sns
import matplotlib.pyplot as plt


import biblio_extractor as bex

np.set_printoptions(precision=4, suppress=True)
pd.set_option("display.max_columns", 10)
pd.set_option("display.max_rows", 20)
pd.set_option("display.min_rows", 10)
# pd.set_option('display.expand_frame_repr', False)

DATASET_FILENAME = Path("results/activities_2022-01-29_16-33-05.csv")


# Loading the CSV file

Now, we load the dataset we'll use in this Notebook. It is generated using [our tool](biblio_extractor.py) in the same repository.
The dataset is a 106 \* 66 matrix, with rows and columns indexed by three levels (using Pandas's `MultiIndex`) as follows:

- level 0 is _class_ (chemical, biological or pharmacological class)
- level 1 is _name_ (compound or activity, a.k.a. keyword)
- level 2 is _kind_ (either _w/o_ for _without_ or _w/_ for _with_)

The matrix is indexed by _two_ disjoint finite sets of keywords:

- **rows**: a set of 53 (chemical) coumpounds = {acridine, triterpene, ...}
- **columns**: a set of 33 (biological, pharmacological) activities = {germination, cytotoxicity, ...}

An example is shown below, restricted to two compounds and two activities.
Note that this is **not** an extract of the whole dataset, the (w/o, w/o) cells being smaller here.

|                   |            |     | germination | germination | cytotoxicity | cytotoxicity |
| ----------------- | ---------- | --- | ----------- | ----------- | ------------ | ------------ |
|                   |            |     | allelopathy | allelopathy | pharmaco     | pharmaco     |
|                   |            |     | w/o         | w/          | w/o          | w/           |
| alkaloid          | acridine   | w/o | 2100        | 32          | 28           | 2104         |
| alkaloid          | acridine   | w/  | 1294        | 11          | 9            | 1296         |
| terpenoid/terpene | triterpene | w/o | 1283        | 11          | 9            | 1285         |
| terpenoid/terpene | triterpene | w/  | 2111        | 32          | 28           | 2115         |

Let $M$ be this matrix, and for each couple of keywords made of a compound and and activity, call $M_{ij} = (c_i, a_j)$, the **ij confusion submatrix**.
Assume that $M_ij$ is of the form :
\begin{bmatrix}
U & V\\
X & Y
\end{bmatrix}

Where :

- $U = (\text{w/o}, \text{w/o})$ is the number of papers that have **neither** the $c_i$ compound **nor** the $a_j$ activity;
- $V = (\text{w/o}, \text{w/})$ is the number of papers that have the $a_j$ activity but **not** the $c_i$ compound;
- $X = (\text{w/}, \text{w/o})$ is the number of papers that have the $c_i$ compound but **not** the $a_j$ activity;
- $Y = (\text{w/}, \text{w/})$ is the number of papers that have **both** the $c_i$ compound **and** the $a_j$ activity.

We avoid the open world hypothesis by restricting the analysis to the paper in the domain $D$,
which is the set of papers that have at least one compound and one activity.
By construction:

- $U + V$ and $X + Y$ are constants for each $c_i$ (whatever the choice of $a_j$) and is the total number of papers in $D$ with the $c_i$ comppound;
- $U + X$ and $V + Y$ are constants for each $a_j$ (whatever the choice of $c_i$) and is the total number of papers in $D$ with the $a_j$ activity.
- each confusion matrix $M_{ij}$ is such that $U + V + X + Y = |D|$ where $|D|$ is _the total number of paper_ under scrutiny.


In [49]:
dataset = pd.read_csv(DATASET_FILENAME, index_col=[0, 1, 2], header=[0, 1, 2])
# dataset.index.names = ["class", "name", "with"]
# dataset.columns.names = ["class", "name", "with"]

all_compounds = set(dataset.index.get_level_values(1))
all_activities = set(dataset.columns.get_level_values(1))

dataset


abiotic                               ...     pharmaco                toxicity       
                                                          antioxidant        drought         metal  ... cytotoxicity sedative       toxicity       
                                                                  w/o     w/     w/o    w/     w/o  ...           w/      w/o    w/      w/o     w/
alkaloid          acridine                            w/o      179092  62176  240191  1077  216324  ...        39229   238785  2483   215604  25664
                                                      w/         2430    266    2694     2    2439  ...         1296     2688     8     2294    402
                  benzylamine                         w/o      180754  62371  242046  1079  218089  ...        40400   240640  2485   217161  25964
                                                      w/          768     71     839     0     674  ...          125      833     6      737    102
                  colchicine                          w/o      175968  62250  237143  1075  213101  ...        39268   235761  2457   213018  25200
...                                                               ...    ...     ...   ...     ...  ...          ...      ...   ...      ...    ...
terpenoid/terpene sesterterpene                       w/          182      7     189     0     189  ...          113      188     1      177     12
                  tetraterpene/carotenoid/xanthophyll w/o      178534  54855  232655   734  208780  ...        40054   230901  2488   208151  25238
                                                      w/         2988   7587   10230   345    9983  ...          471    10572     3     9747    828
                  triterpene                          w/o      177099  61285  237308  1076  213234  ...        38416   235915  2469   212731  25653
                                                      w/         4423   1157    5577     3    5529  ...         2109     5558    22     5167    413

[106 rows x 66 columns]

We can extract the "old" 53\*33 matrix we use in the version version of this software, where we had only the with/with queries.
In our small running example, that would be:

|                   |            | germination | cytotoxicity |
| ----------------- | ---------- | ----------- | ------------ |
|                   |            | allelopathy | pharmaco     |
| alkaloid          | acridine   | 11          | 1296         |
| terpenoid/terpene | triterpene | 32          | 2115         |


In [50]:
with_with_matrix = dataset.xs("w/", level=2).xs("w/", level=2, axis=1)
with_with_matrix


abiotic                           ... pharmaco                                  toxicity
                                                      antioxidant drought metal   uv  salt  ...    wound anticancer cytotoxicity sedative toxicity
alkaloid          acridine                                    266       2   257   80   163  ...       77        147         1296        8      402
                  benzylamine                                  71       0   165   23    80  ...       19         19          125        6      102
                  colchicine                                  192       4    84   20   187  ...      222        171         1257       34      866
                  cyclopeptide                                 57       1   168   16    35  ...       51         45          523        4      171
                  imidazole                                  1082       8  2507  302  1195  ...      460        336         2816      486     1768
...                                                           ...     ...   ...  ...   ...  ...      ...        ...          ...      ...      ...
terpenoid/terpene polyterpene                                   0       0     0    0     0  ...        0          0            0        0        2
                  sesquiterpene                               863      17    72   39    59  ...      124        114         2037       30      341
                  sesterterpene                                 7       0     0    0     3  ...        2          6          113        1       12
                  tetraterpene/carotenoid/xanthophyll        7587     345   592  392   513  ...       97         51          471        3      828
                  triterpene                                 1157       3    51   46    59  ...      203        142         2109       22      413

[53 rows x 33 columns]

We do some sanity checks explained earlier :

- the name (level 2 of rows/columns) are _unique_
- the sum of each confusion submatrixes is _constant_ and is the total number of papers $|D|$, here 243 964.


In [51]:
# sanity check #1
assert dataset.shape == (2 * len(all_compounds), 2 * len(all_activities))
dataset


# sanity check : group by summing on level 2 on both rows and cols produce a matrix of constants : the number of papers
submatrix_sum = dataset.groupby(level=1).sum().groupby(level=1, axis=1).sum()
number_of_papers = np.unique(submatrix_sum.values)
# if the Scopus collection did not evolve during while querying
assert len(number_of_papers) == 1

number_of_papers = number_of_papers[0]
print(f"The domain contains {number_of_papers} papers")


The domain contains 243964 papers


Lets illustrate the content of this table. The **2 by 2 confusion submatrix** about _acridine_ and _cytotoxicity_ is as follows.


In [52]:
acridine_antioxidant_submatrix = dataset.loc[
    (
        "alkaloid",
        "acridine",
    ),
    (
        "pharmaco",
        "cytotoxicity",
    ),
]
print(f"Among {number_of_papers} papers, there are")
for i, j in product(bex.SELECTORS, bex.SELECTORS):
    print(f"{acridine_antioxidant_submatrix.loc[i,j]} papers {i} acridine and {j} cytotoxicity in their keywords")

print("The acridine and cytotoxicity confusion matrix is as follows")
acridine_antioxidant_submatrix


Among 243964 papers, there are
202039 papers w/o acridine and w/o cytotoxicity in their keywords
39229 papers w/o acridine and w/ cytotoxicity in their keywords
1400 papers w/ acridine and w/o cytotoxicity in their keywords
1296 papers w/ acridine and w/ cytotoxicity in their keywords
The acridine and cytotoxicity confusion matrix is as follows


/tmp/ipykernel_73648/2841051283.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  acridine_antioxidant_submatrix = dataset.loc[


,w/o,w/
w/o,202039,39229
w/,1400,1296


abiotic          \
                                                      antioxidant drought   
alkaloid          acridine                                    266       2   
                  benzylamine                                  71       0   
                  colchicine                                  192       4   
                  cyclopeptide                                 57       1   
                  imidazole                                  1082       8   
                  indole                                     1324      53   
                  indolizidine                                  1       0   
                  isoquinoline                                138       2   
                  isoxazole                                   127       2   
                  muscarine                                     2       0   
                  oxazole                                      71       1   
                  phenylethylamine                              3       0   
                  piperidine                                  577       1   
                  purine                                      361      19   
                  putrescine                                  197      59   
                  pyridine                                   1033       7   
                  pyrrolidine                                 769       0   
                  pyrrolizidine                                32       2   
                  quinazoline                                 133       0   
                  quinoline                                   560       4   
                  quinolizidine                                 3       1   
                  spermidine                                  259      43   
                  spermine                                    223      37   
                  thiazole                                    863       4   
                  tropane                                      21       2   
phenolic compound acetophenone                                485       3   
                  anthraquinone                               701       2   
                  biflavonoids                                400       1   
                  flavonoids                                25763     259   
                  isoflavanoids                                 0       0   
                  lignans                                     757       2   
                  naphthoquinone                              371       1   
                  phenol                                    27855     232   
                  phenolic acid                              1825      20   
                  phenylpropanoid                             357      48   
                  stilbene                                   2475       5   
                  tannin                                     5276      48   
                  xanthone                                    481       0   
polyketide        acetogenin                                   20       1   
                  ansamycin                                     4       0   
                  macrolide                                    71       3   
                  polyene                                      74       1   
                  polyether                                    25       1   
                  tetracycline                                466      14   
terpenoid/terpene diterpene                                   427       5   
                  hemiterpene                                  29      24   
                  monoterpene                                 188      50   
                  norisoprenoid                                18       1   
                  polyterpene                                   0       0   
                  sesquiterpene                               863      17   
                  sesterterpene                       

In [ ]:
with_with_total = with_with_matrix.values.sum()
print(
    f"Total number of positive/positive occurences is {with_with_total} for {number_of_papers} papers (average={with_with_total/number_of_papers})"
)


We compute marginal sums on rows and cols


In [ ]:
margin_idx = (bex.CLASS_SYMB, bex.MARGIN_SYMB, bex.SELECTORS[1])
margin_cols = dataset.groupby(level=1).sum().drop_duplicates().reset_index(drop=True)
margin_cols.index = pd.MultiIndex.from_tuples([margin_idx])
margin_cols


In [ ]:
margin_rows = dataset.groupby(level=1, axis=1).sum().iloc[:, 0]
margin_rows.name = margin_idx
margin_rows = pd.DataFrame(margin_rows)
margin_rows


In [ ]:
# me way add those margin to the original dataset
dataset_margins = dataset.copy()
dataset_margins[margin_idx] = margin_rows
dataset_margins = dataset_margins.append(margin_cols).fillna(number_of_papers).astype(int)


In [ ]:
# see https://en.wikipedia.org/wiki/Confusion_matrix
# /!\ quelle jungle /!\


def fowlkes_mallows(arr):
    arr = arr.reshape(2, 2)
    return np.sqrt(arr[1][1] / (arr[1][1] + arr[0][1]) * arr[1][1] / (arr[1][1] + arr[1][0]))


def acc(arr):
    arr = arr.reshape(2, 2)
    return (arr[1][1] + arr[0][0]) / (arr.sum())


def x_score(arr):
    arr = arr.reshape(2, 2)
    return (arr[1][1] + arr[0][0] - arr[0][1] - arr[1][0]) / (arr.sum())


print(fowlkes_mallows(acridine_antioxidant_submatrix.values))
print(acc(acridine_antioxidant_submatrix.values))
print(x_score(acridine_antioxidant_submatrix.values))
acridine_antioxidant_submatrix


In [ ]:
# redimension the values to a 4D array
C, A = len(all_compounds), len(all_activities)
print(C, A)
M = dataset.values.reshape((C, 2, A, 2))
M = np.moveaxis(M, 1, -2)


We obtain the same as submatrix_sum


In [ ]:
# M.sum(axis=(2,3)) or similarly
np.sum(M, axis=(2, 3), keepdims=False)


In [ ]:
# 1D is easier for apply_along_axis
M2 = M.reshape((C * A, 4))

print(M2[0])
print(M[0, 0])
print(acc(M[0, 0]))
print(x_score(M[0, 0]))
print(fowlkes_mallows(M[0, 0]))
print("--")


for fnct in [acc, x_score, fowlkes_mallows]:
    confused = np.apply_along_axis(fnct, 1, M2).reshape((C, A))

    print(f"{fnct.__name__}: min={confused.min()} max={confused.max()} mean={confused.mean()} std={confused.std()}")
# np.apply_over_axes(f, M2, axes=1)
# confused


In [ ]:
confused_df = pd.DataFrame(100 * confused)
confused_df.index = with_with_matrix.index  #  set([(x,y) for x,y,_ in dataset.index.to_list()])
confused_df.columns = with_with_matrix.columns
# confused_df.values = confused
# confused_df


In [ ]:
# TODO : les couleurs par classes

ca = prince.CA(n_components=2, n_iter=5, copy=True, check_input=True, engine="auto", random_state=42, benzecri=False)

for (df, name) in [(with_with_matrix, "Original"), (confused_df, "Confused")]:
    print(f"-------{name}-------")
    ca = ca.fit(df)

    # ca.row_coordinates(df)[:10]
    # ca.column_coordinates(df)[:10]

    pprint(ca.explained_inertia_)
    pprint(ca.col_masses_[:10])
    pprint(ca.eigenvalues_)
    pprint(ca.total_inertia_)

    ax = ca.plot_coordinates(
        X=confused_df,
        ax=None,
        figsize=(12, 12),
        x_component=0,
        y_component=1,
        show_row_labels=True,
        show_col_labels=True,
    )
    plt.show()
